In [1]:
%load_ext autotime


In [2]:
from dask_ml.impute import *
from dask_ml.preprocessing import *
import dask.array as da

import streamlit as st
import numpy as np
import pandas as pd
import random
import ray
ray.init()
# from pycaret.classification import *

2021-05-29 14:29:02,069	INFO services.py:1171 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.3.6',
 'raylet_ip_address': '192.168.3.6',
 'redis_address': '192.168.3.6:6379',
 'object_store_address': '/tmp/ray/session_2021-05-29_14-29-01_572985_143243/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-05-29_14-29-01_572985_143243/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-05-29_14-29-01_572985_143243',
 'metrics_export_port': 56626,
 'node_id': '55fc5627109b6c8b86bd2ffa039dd0f00820e287'}

time: 3.14 s


In [3]:
# chunksize = 10**6
# p = 0.0001
      
wa = [ 'demographics.ageRange.start',
        'demographics.gender',
        'events.confirmed.date',
        'location.country',
        'location.geometry.latitude',
        'location.geometry.longitude',
        # 'variantOfConcern' 
        ]

df = pd.read_csv("../Data/globaldothealth_2021-03-22.csv",
                 header=0, low_memory=False,
                #  skiprows=lambda i: i > 0 and random.random() > p
                )
df = df[wa]
df = df.rename(
    columns={
        'demographics.ageRange.start': 'Age',
        'demographics.gender':'Gender',
        'events.confirmed.date': 'Date',
        'location.country': 'Country',
        'location.geometry.latitude': 'latitude',
        'location.geometry.longitude': 'longitude',
        # 'variantOfConcern': 'KnownVariant'
    })



time: 2min 42s


In [4]:

# df.head()
# df.dtypes

# df['testtarget'] = df['demographics.gender']

# grid = setup(df, target='testtarget', silent=False, verbose=True, profile=True)


time: 1.14 ms


In [5]:
# import dask.dataframe as dd
# df = dd.read_csv("../Data/globaldothealth_2021-03-22.csv",
#                  low_memory=False, 
                 
#                  assume_missing=True,
#                  dtype={
#                      'caseReference.additionalSources': 'object',
#                      'caseReference.sourceEntryId': 'object',
#                      'demographics.nationalities': 'object',
#                      'events.hospitalAdmission.value': 'object',
#                      'genomeSequences': 'object',
#                      'demographics.ethnicity': 'object',
#                      'demographics.occupation': 'object',
#                      'events.hospitalAdmission.date': 'object',
#                      'events.icuAdmission.date': 'object',
#                      'events.icuAdmission.value': 'object',
#                      'location.place': 'object',
#                      'preexistingConditions.values': 'object',
#                      'symptoms.values': 'object',
#                      'events.firstClinicalConsultation.date': 'object',
#                      'transmission.places': 'object'
#                  })

# # %%
# # from dask.distributed import Client

# # client = Client(n_workers=4, threads_per_worker=1)
# # client
# # %%
# # df.head(10)
# # %%
# df = df[wa]
# df = df.rename(
#     columns={
#         'demographics.ageRange.start': 'Age',
#         # 'demographics.gender':'Gender',
#         'events.confirmed.date': 'Date',
#         'location.country': 'Country',
#         'location.geometry.latitude': 'latitude',
#         'location.geometry.longitude': 'longitude',
#         'variantOfConcern': 'KnownVariant'
#     })


time: 170 µs


In [6]:
df.isna().sum()


Age          2284834
Gender       2361701
Date               0
Country       125075
latitude      125075
longitude     125075
dtype: int64

time: 1.69 s


In [7]:


df = df[df.Date == '2020-06-02']
# # %%
# # df = df.head(40)

time: 971 ms


In [8]:
df = df.dropna(
    subset=['latitude', 'longitude']
    )

time: 2.88 ms


In [9]:
df.head(50)


,Age,Gender,Date,Country,latitude,longitude
8,60.000000,Male,2020-06-02,Argentina,-34.59972,-58.38194
9,49.000000,Female,2020-06-02,Argentina,-34.59972,-58.38194
18,34.000000,Female,2020-06-02,Argentina,-34.59972,-58.38194
19,31.000000,Male,2020-06-02,Argentina,-34.59972,-58.38194
21,28.000000,Female,2020-06-02,Argentina,-34.59972,-58.38194
23,44.000000,Male,2020-06-02,Argentina,-34.59972,-58.38194
24,41.000000,Male,2020-06-02,Argentina,-34.59972,-58.38194
28,25.000000,Male,2020-06-02,Argentina,-34.59972,-58.38194
32,34.000000,Male,2020-06-02,Argentina,-34.59972,-58.38194
33,0.916667,Female,2020-06-02,Argentina,-34.59972,-58.38194


time: 18.8 ms


In [10]:
df['dailytotals'] = 1
df = df[['Country','latitude','longitude','Date','dailytotals']]

df = df.groupby(['Country','latitude','longitude','Date']).sum()


subset=['latitude', 'longitude']


time: 21.6 ms


In [11]:
df.tail(10)

dailytotals
Country        latitude  longitude  Date                   
United Kingdom 53.178836 -3.086491  2020-06-02            1
               53.283333 -4.333333  2020-06-02            3
               54.370020 -2.952140  2020-06-02         1493
               55.583333 -3.833333  2020-06-02            8
               55.882067 -4.320724  2020-06-02            1
               55.909167 -3.084444  2020-06-02            2
               56.027520 -3.843156  2020-06-02            4
               56.250000 -3.200000  2020-06-02           35
               57.228000 -2.988000  2020-06-02            3
United States  37.090200 -95.712900 2020-06-02        16207

time: 16.4 ms


In [12]:
df = df.reset_index()

time: 2.09 ms


In [13]:
import numpy as np
# import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

access_token = 'pk.eyJ1IjoiZ3lhc2lzIiwiYSI6ImNrb3d4MXIxczBhb3YyeG9hOXg3dWoxZmcifQ.bshWQipaw_Jj9OxhxwSDog'
px.set_mapbox_access_token(access_token)


time: 97.6 ms


In [14]:
df.head(10)

,Country,latitude,longitude,Date,dailytotals
0,Argentina,-43.30000,-65.10000,2020-06-02,1
1,Argentina,-40.80000,-63.00000,2020-06-02,5
2,Argentina,-38.95735,-68.04553,2020-06-02,10
3,Argentina,-34.59972,-58.38194,2020-06-02,382
4,Argentina,-31.41667,-64.18333,2020-06-02,1
5,Argentina,-27.78444,-64.26694,2020-06-02,1
6,Argentina,-27.48333,-58.81667,2020-06-02,2
7,Argentina,-27.45139,-58.98667,2020-06-02,8
8,Argentina,-24.78833,-65.41056,2020-06-02,8
9,Brazil,-30.81180,-53.89830,2020-06-02,3


time: 5.02 ms


In [15]:
fig2 = px.scatter_mapbox(
    df, 
    lat="latitude",
    lon="longitude",
    # size = "dailytotals", 
    size_max=50,
    mapbox_style = 'dark', 
    zoom=1,
    # animation_group="Date",
    # animation_frame="Date"
    
    
)
# fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1
# fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 1
# fig.layout.coloraxis.showscale = False
# fig.layout.sliders[0].pad.t = 10
# fig.layout.updatemenus[0].pad.t = 10

time: 327 ms


In [16]:
fig2.show()


time: 43.4 ms


In [17]:
fig2 = px.scatter_mapbox(
    df, 
    lat="latitude",
    lon="longitude",
    size = "dailytotals", 
    size_max=50,
    mapbox_style = 'dark', 
    zoom=1,
    # animation_group="Date",
    # animation_frame="Date"
    
    
)
# fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1
# fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 1
# fig.layout.coloraxis.showscale = False
# fig.layout.sliders[0].pad.t = 10
# fig.layout.updatemenus[0].pad.t = 10

time: 52.3 ms


In [18]:
fig2.show()


time: 7.43 ms


In [19]:
fig2 = px.scatter_mapbox(
    df, 
    lat="latitude",
    lon="longitude",
    size = "dailytotals", 
    color = "dailytotals",
    size_max=50,
    mapbox_style = 'dark', 
    zoom=1,
    # animation_group="Date",
    # animation_frame="Date"
    
    
)
# fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1
# fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 1
# fig.layout.coloraxis.showscale = False
# fig.layout.sliders[0].pad.t = 10
# fig.layout.updatemenus[0].pad.t = 10

time: 54 ms


In [20]:
fig2.show()


time: 10.9 ms


In [21]:
fig2 = px.scatter_mapbox(
    df, 
    lat="latitude",
    lon="longitude",
    size = "dailytotals", 
    color = "dailytotals",
    size_max=200,
    mapbox_style = 'dark', 
    zoom=1,
    # animation_group="Date",
    # animation_frame="Date"
    
    
)
# fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1
# fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 1
# fig.layout.coloraxis.showscale = False
# fig.layout.sliders[0].pad.t = 10
# fig.layout.updatemenus[0].pad.t = 10

time: 52.6 ms


In [22]:
fig2.show()


time: 8.28 ms


In [25]:
fig2 = px.scatter_mapbox(
    df, 
    lat="latitude",
    lon="longitude",
    size = "dailytotals", 
    color = "dailytotals",
    size_max=200,
    mapbox_style = 'dark', 
    zoom=10,
    # animation_group="Date",
    # animation_frame="Date"
    
    
)
# fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1
# fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 1
# fig.layout.coloraxis.showscale = False
# fig.layout.sliders[0].pad.t = 10
# fig.layout.updatemenus[0].pad.t = 10

time: 35.9 ms


In [26]:
fig2.show()


time: 9.37 ms


In [27]:
fig2 = px.scatter_mapbox(
    df, 
    lat="latitude",
    lon="longitude",
    size = "dailytotals", 
    color = "dailytotals",
    size_max=200,
    mapbox_style = 'dark', 
    zoom=0.2,
    # animation_group="Date",
    # animation_frame="Date"
    
    
)
# fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1
# fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 1
# fig.layout.coloraxis.showscale = False
# fig.layout.sliders[0].pad.t = 10
# fig.layout.updatemenus[0].pad.t = 10

time: 50.5 ms


In [28]:
fig2.show()


time: 8.39 ms


In [29]:
fig2 = px.scatter_mapbox(
    df, 
    lat="latitude",
    lon="longitude",
    size = "dailytotals", 
    color = "dailytotals",
    size_max=100,
    mapbox_style = 'dark', 
    zoom=0.2,
    # animation_group="Date",
    # animation_frame="Date"
    
    
)
# fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1
# fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 1
# fig.layout.coloraxis.showscale = False
# fig.layout.sliders[0].pad.t = 10
# fig.layout.updatemenus[0].pad.t = 10

time: 49.1 ms


In [30]:
fig2.show()


time: 35.7 ms
